<a href="https://colab.research.google.com/github/DDDCR/Relationships_extraction/blob/main/relationships_extraction_follow_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!pip install pandas
!pip install bs4

In [5]:
import re
import csv
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
from spacy.tokens import Span

import urllib.request
from bs4 import BeautifulSoup

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [6]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################

  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text

      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text

      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text

      ## chunk 5
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [13]:
text = '''i have 2+ years of non-internship professional software development experience Programming experience with at least one modern language such as Java, C++, or C# including object-oriented design.
  1+ years of experience contributing to the architecture and design (architecture, design patterns, reliability and scaling) of new and current systems.
  Bachelor / MS Degree in Computer Science. Preferably a PhD in data science. 8+ years of professional experience in software development.
  2+ years of experience in project management. Experience in mentoring junior software engineers to improve their skills, and make them more effective, product software engineers.
  Experience in data structures, algorithm design, complexity analysis, object-oriented design.
  3+ years experience in at least one modern programming language such as Java, Scala, Python, C++, C#
  Experience in professional software engineering practices & best practices for the full software development life cycle,
  including coding standards, code reviews, source control management, build processes, testing, and operations Experience in communicating with users,
  other technical teams, and management to collect requirements, describe software product features, and technical designs.
  Experience with building complex software systems that have been successfully delivered to customers Proven ability to take a project from scoping requirements through actual launch of the project,
  with experience in the subsequent operation of the system in production'''

In [16]:
sentences = [[i] for i in nlp(text).sents]
sentences

[[i have 2+ years of non-internship professional software development experience Programming experience with at least one modern language such as Java, C++, or C# including object-oriented design.
    ],
 [1+ years of experience contributing to the architecture and design (architecture, design patterns, reliability and scaling) of new and current systems.
    ],
 [Bachelor / MS Degree in Computer Science.],
 [Preferably a PhD in data science.],
 [8+ years of professional experience in software development.
    ],
 [2+ years of experience in project management.],
 [Experience in mentoring junior software engineers to improve their skills, and make them more effective, product software engineers.
    ],
 [Experience in data structures, algorithm design, complexity analysis, object-oriented design.
    ],
 [3+ years experience in at least one modern programming language such as Java, Scala, Python, C++, C#
    Experience in professional software engineering practices & best practices for 

In [17]:
myheaders = ['sentences']
myvalues = sentences
filename = 'text_sentences.csv'
with open(filename, 'w',newline='') as myfile:
    writer = csv.writer(myfile)
    writer.writerow(myheaders)
    writer.writerows(myvalues)

In [23]:
csv_sentences = pd.read_csv("/content/text_sentences.csv")

In [25]:
entity_pairs = []

for i in tqdm(csv_sentences["sentences"]):
  entity_pairs.append(get_entities(i))

100%|██████████| 10/10 [00:00<00:00, 112.81it/s]


In [26]:
entity_pairs

[['i', 'oriented C design'],
 ['', 'new design systems'],
 ['', 'Bachelor Computer Science'],
 ['', 'Preferably data science'],
 ['', 'professional software development'],
 ['', '2 project management'],
 ['junior software them', 'junior software skills'],
 ['', 'data structures'],
 ['+ years experience', 'technical software product features'],
 ['complex software that', 'subsequent  production']]

In [8]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object
  matcher = Matcher(nlp.vocab)

  #define the pattern
  pattern = [{'DEP':'ROOT'},
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},
            {'POS':'ADJ','OP':"?"}]

  matcher.add("matching_1",[pattern])

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]]

  return(span.text)

In [9]:
get_relation("i finished all my homework")

'finished'

In [27]:
relations = [get_relation(i) for i in tqdm(csv_sentences["sentences"])]

100%|██████████| 10/10 [00:00<00:00, 106.17it/s]


In [28]:
relations

['have',
 'years of',
 'Degree in',
 'PhD in',
 'years of professional',
 'years of',
 'Experience in',
 'Experience in',
 'describe',
 'Proven']

In [29]:
pd.Series(relations).value_counts()[:50]

years of                 2
Experience in            2
have                     1
Degree in                1
PhD in                   1
years of professional    1
describe                 1
Proven                   1
dtype: int64